In [1]:
import eia_model as em
import parms as parms
import plotly.plotly as py
from plotly.graph_objs import *
from plotly import tools
from datetime import datetime, timedelta

# Create a weather forecast for SeaTac
http://openweathermap.org/

In [2]:
# Use GetWeatherForecast to pull SeaTac weather forecast from OpenWeatherMap
forecast = em.GetWeatherForecast(parms.owm_api, parms.city_id, parms.units)

In [3]:
# create interpolated hourly forecast and 3 hour forecast to compare
temps_one_hour = forecast.df_hr[:28]
temps_three_hour = forecast.df[:10]

In [4]:
# plot hour against 3 hour
fig = tools.make_subplots(rows=2)
fig.append_trace(Scatter(
        x=temps_one_hour.index,
        y=temps_one_hour['temp'],
        name='one hour',
        mode = 'lines+markers'), 1, 1
)
fig.append_trace(Scatter(
        x=temps_three_hour.index,
        y=temps_three_hour['temp'],
        name='three hour', mode = 'lines+markers'), 2, 1
)
py.iplot(fig, filename='SeaTac Weather Forecast')

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]



# Pull SeaTac weather station history from NCDC
http://www7.ncdc.noaa.gov/rest/

In [5]:
# create date index to pull weather history
date2 = temps_one_hour.index.max()- timedelta(days=7)
date1 = date2 - timedelta(hours=28)

In [6]:
# function to convert datetime to string %Y-%m%-d %H:%M:%S format for NCDC
def convert_date(date):
    return datetime.strftime(date, '%Y-%m-%d %H:%M:%S')

In [7]:
# Use GetWeather to pull hourly data from SeaTac weather station
start, end = convert_date(date1), convert_date(date2)
history = em.GetWeatherHistory(parms.ncdc_api, parms.station, parms.variable, start, end)

In [8]:
# plot hour against last week
fig = tools.make_subplots(rows=2)
fig.append_trace(Scatter(
        x=temps_one_hour.index,
        y=temps_one_hour['temp'],
        name='one hour',
        mode = 'lines+markers'), 1, 1
)
fig.append_trace(Scatter(
        x=history.df.index,
        y=history.df['temp'],
        name='last week',
        mode = 'lines+markers'), 2, 1
)
py.iplot(fig, filename='SeaTac Weather Forecast vs Last Week')

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]



# Import Seattle City Light load data
http://www.eia.gov/opendata/date

In [9]:
# Use GetEnergy to pull hourly data from EIA API for Seattle City Light
start, end = convert_date(date1), convert_date(date2)
scl_load = em.GetEnergy(parms.eia_api, parms.sid,'H',start, end)

In [10]:
#Plot load data against weather history
trace1 = Scatter(
    x=scl_load.df.index,
    y=scl_load.df['values'],
    mode = 'lines+markers',
    name='scl_load',
)

trace2 = Scatter(
    x=history.df.index,
    y=history.df['temp'],
    mode = 'lines+markers',
    name = 'temp',
    yaxis='y2'
)

data = [trace1, trace2]

layout = Layout(
    title='SCL Load vs SeaTac Temperature',
    yaxis=dict(
        title='Load MWhs',
        titlefont=dict(
            color='rgb(0,102,204)'
        ),
        tickfont=dict(
            color='rgb(0,102,204)'
        )
    ),
    yaxis2=dict(
        title='Temp F',
        titlefont=dict(
            color='rgb(255,140,0'
        ),
        tickfont=dict(
            color='rgb(255,140,0'
        ),
        overlaying='y',
        side='right'
    )
)
fig = Figure(data=data, layout=layout)
py.iplot(fig, filename='multiple-axes-double')